In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import glob 

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom

In [2]:
outpath = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/'
local_modpath = '/home/birniece/Documents/data/ANPERC_Lab/STYLOLITES/StyloliteModels'

In [3]:
mod_path = './inputs/StyloliteModels/'
rec_file = './inputs/gridded_recs.dat'
src_file = './inputs/central_source.dat'

In [4]:
# PANCAKE ONES 
shaheen_cp_loc = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites'

# 'FAKE' Shaheen Path
shaheen_proj_loc = '/scratch/birniece/SOFI_StylolitesRockBlock/Stylolites'

# SOFI Shaheen Path
sofimaster = '/project/k1056/SOFI_FDmodelling/sofi3d/bin/sofi3D'

In [5]:
def slurmfilecreation(slurmfile, expname, sofimaster, jsonloc, joutloc, write=True):
    slurmstr='''#!/bin/bash -l
#SBATCH --job-name=SOFI_Stylolites_%s
#SBATCH --nodes=32
#SBATCH --partition=workq
#SBATCH -A k1056
#SBATCH --time=00:45:00
#SBATCH --error=sofi.%%J.err 
#SBATCH --output=sofi.%%J.out

#OpenMP settings:
export OMP_NUM_THREADS=1

# RUN MODELLING JOB
srun --hint=nomultithread --ntasks=1000 %s %s.json > %s.jout
'''%(expname, sofimaster, jsonloc, joutloc)
    
    if write:
        text_file = open(slurmfile, "wt")
        text_file.writelines(slurmstr)
        text_file.close()

# MODEL PARAMETERS - Get these from the script that made the models

In [6]:
# TIME PARAMETERS (in seconds!)
dt = 5e-8
tdur = 0.0005

In [7]:
# SPATIAL PARAMETERS
dx = dy = dz = 0.001
n_xzy = (460, 460, 460)
d_xzy = [dx, dz, dy]

print(n_xzy[0]*dx,n_xzy[1]*dx,n_xzy[2]*dx)

0.46 0.46 0.46


In [8]:
# BOUNDARIES & FREE SURFACE
nbounds = 30
free_surface = False

In [9]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur
snap_step = 100*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 2

# SEISMOGRAM PARAMETERS
smgrm_ndt = 10

In [10]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
t_str = get_time_str(dt, tdur)

bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=False)

src_str = get_source_str(sfile=src_file, 
                         multisource=0)




## LOOP OVER ALL VELOCITY MODELS

In [11]:
model_files = glob.glob(local_modpath+'/*.vp')


In [12]:
for mod_file in model_files:
    expname = mod_file.split('/')[-1][:-3]
    # print(expname)
    mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              ' ',
                              moddir=os.path.join(mod_path,expname),
                              np_xzy=[10,10,10]
                             )
    # Make Monitor String
    monitor_strs = get_monitor_str(tsnap_params, 
                                   smgrm_ndt, 
                                   expname, 
                                   sbsmp_xyz=snap_sbsmp_xyz,
                                   snap=True
                                  )
    # Write SOFI FILE
    sofi_json_file = os.path.join(shaheen_cp_loc, expname+'.json')
    shaheen_sofi_json_file = os.path.join(shaheen_proj_loc, expname+'.json')
    print(sofi_json_file)
    write_SOFIjsonParams(default_strs, 
                         monitor_strs,  
                         t_str, 
                         mod_strs, 
                         src_str, 
                         bndry_str,
                         sofi_json_file)

    # WRITE SLURM FILE
    slurmfile = os.path.join(shaheen_cp_loc, 'submit_'+expname+'.slurm')
    slurmfilecreation(slurmfile, 
                      expname=expname, 
                      sofimaster=sofimaster, 
                      jsonloc=shaheen_sofi_json_file[:-5], 
                      joutloc=shaheen_sofi_json_file[:-5],)
    print(' ')

/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/StraightStylo_w05_pc80.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/InclinedStylo_a45_pc90.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/SineStylo_h20_pc70.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/InclinedStylo_a-45_pc90.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/InclinedStylo_a-35_pc90.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/SineStylo_h05_pc70.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/InclinedStylo_a-45_pc70.json
 
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/SineStylo_h10_pc80.json
 
/home/birniece/Documents/

# MAKE SINGLE SUBMIT SCRIPT

In [13]:
slurmfiles = []
for file in glob.glob(shaheen_cp_loc+"/*.slurm"):
    slurmfiles.append(file.split('/')[-1])


In [14]:
bashsubmitfile = os.path.join(shaheen_cp_loc, 'submit_ALL.sh')

bashsubmitstring = '''#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

'''

for slurmf in slurmfiles:
    bashsubmitstring+='sbatch %s\n'%slurmf 
print(bashsubmitstring)

text_file = open(bashsubmitfile, "wt")
text_file.writelines(bashsubmitstring)
text_file.close()

#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

sbatch submit_InclinedStylo_a45_pc70.slurm
sbatch submit_InclinedStylo_a15_pc90.slurm
sbatch submit_StraightStylo_w05_pc80.slurm
sbatch submit_StraightStylo_w02_pc80.slurm
sbatch submit_InclinedStylo_a25_pc80.slurm
sbatch submit_SineStylo_h15_pc80.slurm
sbatch submit_SineStylo_h15_pc70.slurm
sbatch submit_InclinedStylo_a-35_pc70.slurm
sbatch submit_InclinedStylo_a-15_pc90.slurm
sbatch submit_InclinedStylo_a05_pc70.slurm
sbatch submit_SineStylo_h05_pc80.slurm
sbatch submit_InclinedStylo_a-35_pc80.slurm
sbatch submit_InclinedStylo_a25_pc90.slurm
sbatch submit_InclinedStylo_a-05_pc90.slurm
sbatch submit_InclinedStylo_a35_pc70.slurm
sbatch submit_InclinedStylo_a-25_pc70.slurm
sbatch submit_InclinedStylo_a45_pc90.slurm
sbatch submit_PinchOutStylo_pc90.slurm
sbatch submit_InclinedStylo_a-45_pc80.slurm
sbatch submit_StraightStylo_w01_pc70.slurm
sbatch submit_StraightStylo_w04_pc70.slurm
sbatch submi